In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse import dia_matrix, csr_matrix, csc_matrix, tril, identity, triu
from scipy.sparse.linalg import spsolve_triangular, spsolve

### Functions

In [ ]:
def GF_table(bc_left: str, bc_right: str, L, truncation):
    # returns the eigenvalue beta_m and the norm Nx

    m = np.arange(1, truncation+1)
    if (bc_left == 'D' and bc_right == 'D'): #11
        beta_m = m*np.pi/L
        # X_m = np.sin(beta_m*x)
        Nx = L/2
        return beta_m, Nx
    
    if (bc_left == 'D' and bc_right == 'N'): #12
        beta_m = (2*m-1)*np.pi/(2*L)
        # X_m = np.sin(beta_m*x)
        Nx = L/2
        return beta_m, Nx
    
    if (bc_left == 'N' and bc_right == 'D'): #21
        beta_m = (2*m-1)*np.pi/(2*L)
        # X_m = np.sin(beta_m*x)
        Nx = L/2
        return beta_m, Nx
    

def GF_function(x, y, x_s, y_s, Lx, Ly, bc_e: str, bc_n: str, bc_w: str, bc_s: str, truncation = 50):
    # x = np.arange(0, Lx, step)
    # y = np.arange(0, Ly, step)
    beta_m, Nx = GF_table(bc_e, bc_w, Lx, truncation)
    theta_n, Ny = GF_table(bc_s, bc_n, Ly, truncation)
    summation_array = np.zeros(truncation*truncation)
    
    for m in range(1, truncation+1):
        i = m-1

        X_m = np.sin(x*beta_m[i])
        X_m_s = np.sin(x_s*beta_m[i])
        # print("X_m_s: " + str(X_m_s))
        for n in range(1, truncation+1):
            j= n-1

            Y_n = np.sin(y*theta_n[j])
            Y_n_s = np.sin(y_s*theta_n[j])
            # print("X_m_s: " + str(Y_n_s))

            fraction = 1/((beta_m[i]*beta_m[i]) + (theta_n[j]*theta_n[j]))
            X_mult = (X_m*X_m_s)/Nx
            Y_mult = (Y_n*Y_n_s)/Ny

            summation_array[i*truncation + j] = fraction * X_mult * Y_mult
    
    sum = np.sum(summation_array)
    return sum


def GF_1D_function(x, x_s, Length, bc_in: str, bc_out: str, truncation = 50):
    beta_m, Nx = GF_table(bc_in, bc_out, Length, truncation)
    
    summation_array = np.zeros(truncation)
    
    for m in range(1, truncation+1):
        i = m-1

        X_m = np.sin(x*beta_m[i])
        X_m_s = np.sin(x_s*beta_m[i])
        fraction = 1/((beta_m[i]*beta_m[i]))
        X_mult = (X_m*X_m_s)/Nx

        summation_array[i] = fraction * X_mult
    
    sum = np.sum(summation_array)
    return sum


def return_GF_matrix(x, y, x_s_start, x_s_end, y_s_start, y_s_end, Lx, Ly, bc_e: str, bc_n: str, bc_w: str, bc_s: str, n_step_s = 20):
    x_s = np.linspace(x_s_start, x_s_end, n_step_s)
    y_s = np.linspace(y_s_start, y_s_end, n_step_s)

    GF_matrix = np.zeros((n_step_s, n_step_s))

    for i in range(n_step_s):
        for j in range(n_step_s):
            GF_matrix[i, j] = GF_function(x, y, x_s[i], y_s[j], Lx, Ly, bc_e=bc_e, bc_n=bc_n, bc_w=bc_w, bc_s=bc_s)

    return GF_matrix


def return_GF_1D_array(x, x_s_start, x_s_end, Lx, bc_in: str, bc_out: str, n_step_s = 20):
    x_s = np.linspace(x_s_start, x_s_end, n_step_s)

    GF_matrix = np.zeros(n_step_s)

    for i in range(n_step_s):
            GF_matrix[i] = GF_1D_function(x, x_s[i], Lx, bc_in = bc_in, bc_out = bc_out)

    return GF_matrix


def HeatEq_2D_point(x, y, omega, lambda_v, Lx, Ly, x_s_start, x_s_end, y_s_start, y_s_end, bc_e: str, bc_n: str, bc_w: str, bc_s: str, T, q, n_step_s = 20):
    x_s = np.linspace(x_s_start, x_s_end, n_step_s)
    y_s = np.linspace(y_s_start, y_s_end, n_step_s)

    # 2D GF integral over surface
    GF_S = return_GF_matrix(x, y, x_s_start, x_s_end, y_s_start, y_s_end, Lx, Ly, bc_e, bc_n, bc_w, bc_s) # 2D GF(x_s, y_s)
    integral_GF_ys = np.trapz(GF_S*omega, y_s, axis=0) #over y
    integral_GF_ys_xs = np.trapz(integral_GF_ys, x_s) #over x
    integral_GF_ys_xs = integral_GF_ys_xs * (1/lambda_v)

    # 1D GF as a function of xs and ys
    GF_x = return_GF_1D_array(x, x_s_start, x_s_end, Lx, bc_e, bc_w) # 1D GF(x_s)
    GF_y = return_GF_1D_array(y, y_s_start, x_s_end, Ly, bc_s, bc_n) # 1D GF(y_s)

    # East integral
    if (bc_e == "D"): # Dirichlet
        dGdx_E = np.zeros(n_step_s)
        dGdx_E[1:] = np.diff(GF_y) / np.diff(x_s)
        integral_GF_E = np.trapz(dGdx_E*T, y_s) #over y
    elif (bc_e == "N"): # Neumann
        dTdx_E = np.zeros(n_step_s)
        dTdx_E = np.dot(GF_x, q)/lambda_v
        integral_GF_E = np.trapz(dTdx_E, y_s)

    # North integral
    if (bc_n == "D"): # Dirichlet
        GF_x_N = GF_x[::-1] # swap because of line direction
        dGdy_N = np.zeros(n_step_s)
        dGdy_N[1:] = np.diff(GF_x_N) / np.diff(y_s)
        integral_GF_N = np.trapz(dGdy_N*T, x_s) #over x
    elif (bc_n == "N"): # Neumann
        GF_y_N = GF_y[::-1] # swap because of line direction
        dTdy_N = np.zeros(n_step_s)
        dTdy_N = np.dot(GF_y_N, q)/lambda_v
        integral_GF_N = np.trapz(dTdy_N, x_s)
        
    # West integral
    if (bc_w == "D"): # Dirichlet
        GF_y_W = GF_y[::-1] # swap because of line direction
        dGdx_W = np.zeros(n_step_s)
        dGdx_W[1:] = np.diff(GF_y_W) / np.diff(x_s)
        integral_GF_W = np.trapz(dGdx_W*T, y_s) #over y
    elif (bc_w == "N"): # Neumann
        GF_x_W = GF_x[::-1] # swap because of line direction
        dTdx_W = np.zeros(n_step_s)
        dTdx_W = np.dot(GF_x_W, q)/lambda_v
        integral_GF_W = np.trapz(dTdx_W, y_s) 
        
    # South integral
    if (bc_s == "D"): # Dirichlet
        dGdy_S = np.zeros(n_step_s)
        dGdy_S[1:] = np.diff(GF_x) / np.diff(y_s)
        integral_GF_S = np.trapz(dGdy_S*T, x_s) #over x
    elif (bc_s == "N"): # Neumann
        dTdy_S = np.zeros(n_step_s)
        dTdy_S = np.dot(GF_y, q)/lambda_v
        integral_GF_S = np.trapz(dTdy_S, x_s)
        
    T_xy = integral_GF_ys_xs + integral_GF_E + integral_GF_N + integral_GF_W + integral_GF_S
    return T_xy


### GF plot

In [ ]:
# test plot
Lx = Ly = 1
x_s = y_s = Lx/2
plot_GF = np.zeros((20, 20))
x = np.arange(0, Lx, 0.05)
y = np.arange(0, Ly, 0.05)
for i in range(20):
    for j in range(20):
        plot_GF[i, j] = GF_function(x[i], y[j], x_s, y_s, Lx, Ly, bc_e = "N", bc_n = "N", bc_w = "D", bc_s = "D")
        

# Plot the heatmap
plt.imshow(plot_GF, cmap='jet', interpolation='nearest', origin = 'lower')
plt.colorbar()  # Add a color bar to show the color scale
plt.title('Heatmap of GF')
plt.xlabel('Column Index')
plt.ylabel('Row Index')
plt.show()

### Stage 2

In [ ]:
# 2
omega = 5
lambda_v = 1
Lx = Ly = 1

# boundary values (T or q)
Temp = 10
q = 3


#### 3 points in the domain

In [ ]:
# SOURCE UNIFORM
# source surface area
x_s_start = 0
x_s_end = Lx
y_s_start = 0
y_s_end = Ly

# x_s and y_s array for integral calculation
n_step_s = 20
step_s_x = (x_s_end-x_s_start)/n_step_s
step_s_y = (y_s_end-y_s_start)/n_step_s
x_s = np.linspace(x_s_start, x_s_end, n_step_s)
y_s = np.linspace(y_s_start, y_s_end, n_step_s)

# Homogenoeus dirichlet everywhere
T_xy1_dirichlet_unif = HeatEq_2D_point(0.1, 0.1, omega, lambda_v, Lx, Ly, x_s_start, x_s_end, y_s_start, y_s_end, "D", "D", "D", "D", Temp, q)
T_xy2_dirichlet_unif = HeatEq_2D_point(0.5, 0.1, omega, lambda_v, Lx, Ly, x_s_start, x_s_end, y_s_start, y_s_end, "D", "D", "D", "D", Temp, q)
T_xy3_dirichlet_unif = HeatEq_2D_point(0.7, 0.3, omega, lambda_v, Lx, Ly, x_s_start, x_s_end, y_s_start, y_s_end, "D", "D", "D", "D", Temp, q)

# Homogeneus Neumann + Dirichlet
T_xy1_neudir_unif = HeatEq_2D_point(0.1, 0.1, omega, lambda_v, Lx, Ly, x_s_start, x_s_end, y_s_start, y_s_end, "N", "N", "D", "D", Temp, q)
T_xy2_neudir_unif = HeatEq_2D_point(0.1, 0.1, omega, lambda_v, Lx, Ly, x_s_start, x_s_end, y_s_start, y_s_end, "N", "N", "D", "D", Temp, q)
T_xy3_neudir_unif = HeatEq_2D_point(0.1, 0.1, omega, lambda_v, Lx, Ly, x_s_start, x_s_end, y_s_start, y_s_end, "N", "N", "D", "D", Temp, q)


# SOURCE SMALL REGION
# source surface area
x_s_start = Lx/3
x_s_end = 2*x_s_start
y_s_start = Ly/3
y_s_end = 2*y_s_start

# x_s and y_s array for integral calculation
n_step_s = 20
step_s_x = (x_s_end-x_s_start)/n_step_s
step_s_y = (y_s_end-y_s_start)/n_step_s
x_s = np.linspace(x_s_start, x_s_end, n_step_s)
y_s = np.linspace(y_s_start, y_s_end, n_step_s)

# Homogenoeus dirichlet everywhere
T_xy1_dirichlet_small = HeatEq_2D_point(0.1, 0.1, omega, lambda_v, Lx, Ly, x_s_start, x_s_end, y_s_start, y_s_end, "D", "D", "D", "D", Temp, q)
T_xy2_dirichlet_small = HeatEq_2D_point(0.5, 0.1, omega, lambda_v, Lx, Ly, x_s_start, x_s_end, y_s_start, y_s_end, "D", "D", "D", "D", Temp, q)
T_xy3_dirichlet_small = HeatEq_2D_point(0.7, 0.3, omega, lambda_v, Lx, Ly, x_s_start, x_s_end, y_s_start, y_s_end, "D", "D", "D", "D", Temp, q)

# Homogeneus Neumann + Dirichlet
T_xy1_neudir_small = HeatEq_2D_point(0.1, 0.1, omega, lambda_v, Lx, Ly, x_s_start, x_s_end, y_s_start, y_s_end, "N", "N", "D", "D", Temp, q)
T_xy2_neudir_small = HeatEq_2D_point(0.1, 0.1, omega, lambda_v, Lx, Ly, x_s_start, x_s_end, y_s_start, y_s_end, "N", "N", "D", "D", Temp, q)
T_xy3_neudir_small = HeatEq_2D_point(0.1, 0.1, omega, lambda_v, Lx, Ly, x_s_start, x_s_end, y_s_start, y_s_end, "N", "N", "D", "D", Temp, q)


### Whole field

In [ ]:
# SOURCE UNIFORM
# source surface area
x_s_start = 0
x_s_end = Lx
y_s_start = 0
y_s_end = Ly

# x_s and y_s array for integral calculation
n_step_s = 10
step_s_x = (x_s_end-x_s_start)/n_step_s
step_s_y = (y_s_end-y_s_start)/n_step_s
x_s = np.linspace(x_s_start, x_s_end, n_step_s)
y_s = np.linspace(y_s_start, y_s_end, n_step_s)

# x y grid
x = np.linspace(0, Lx, n_step_s)
y = np.linspace(0, Ly, n_step_s)

# Homogenoeus dirichlet everywhere
T_field_dirichlet_unif = np.zeros((n_step_s, n_step_s))
for i in range(len(x)):
    for j in range(len(x)):
        T_field_dirichlet_unif[i, j] = HeatEq_2D_point(x[i], y[j], omega, lambda_v, Lx, Ly, x_s_start, x_s_end, y_s_start, y_s_end, "D", "D", "D", "D", Temp, q)

# Homogeneus Neumann + Dirichlet
T_field_dirichlet_unif = np.zeros((n_step_s, n_step_s))
for i in range(len(x)):
    for j in range(len(x)):
        T_field_dirichlet_unif[i, j] = HeatEq_2D_point(x[i], y[j], omega, lambda_v, Lx, Ly, x_s_start, x_s_end, y_s_start, y_s_end, "N", "N", "D", "D", Temp, q)


# SOURCE SMALL REGION
# source surface area
x_s_start = Lx/3
x_s_end = 2*x_s_start
y_s_start = Ly/3
y_s_end = 2*y_s_start

# x_s and y_s array for integral calculation
n_step_s = 20
step_s_x = (x_s_end-x_s_start)/n_step_s
step_s_y = (y_s_end-y_s_start)/n_step_s
x_s = np.linspace(x_s_start, x_s_end, n_step_s)
y_s = np.linspace(y_s_start, y_s_end, n_step_s)

# Homogenoeus dirichlet everywhere
T_field_dirichlet_small = np.zeros((n_step_s, n_step_s))
for i in range(len(x)):
    for j in range(len(x)):
        T_field_dirichlet_small[i, j] = HeatEq_2D_point(x[i], y[j], omega, lambda_v, Lx, Ly, x_s_start, x_s_end, y_s_start, y_s_end, "D", "D", "D", "D", Temp, q)

# Homogeneus Neumann + Dirichlet
T_field_dirichlet_small = np.zeros((n_step_s, n_step_s))
for i in range(len(x)):
    for j in range(len(x)):
        T_field_dirichlet_small[i, j] = HeatEq_2D_point(x[i], y[j], omega, lambda_v, Lx, Ly, x_s_start, x_s_end, y_s_start, y_s_end, "N", "N", "D", "D", Temp, q)


### Stage 3

In [ ]:
class SteadyHeat2D:
    def __init__(self, Lx, Ly, dimX, dimY):
        self.l = Lx #lunghezza rettangolo
        self.h = Ly
        self.dimX = dimX #quante divisioni
        self.dimY = dimY

        self.dx = Lx/dimX
        self.dy = Ly/dimY

        self.A = np.identity(self.dimX*self.dimY)
        self.set_inner()
        self.b = np.zeros([self.dimX*self.dimY])
        
    
    # build the linear system
    def set_inner(self):
        for i in range(self.dimX+1, (self.dimX*self.dimY)-self.dimX-1, self.dimX): # the start of each row of inner nodes 
            for j in range(self.dimX-2): # loops through all inner nodes in that row 
                k = i+j
                # builds the matrix like in scicomplab, so each row
                self.A[k][k] = -2 * (1/(self.dx*self.dx) + 1/(self.dy*self.dy)) # central node
                self.A[k][k-1] = 1/(self.dx*self.dx) # side nodes
                self.A[k][k+1] = 1/(self.dx*self.dx)
                self.A[k][k - self.dimX] = 1/(self.dy*self.dy) # upper lower nodes
                self.A[k][k + self.dimX] = 1/(self.dy*self.dy)

    # south
    def set_south(self, bc_type, T_d=0.0, q=0.0, alpha = 0.0, T_inf=0.0):
        if (bc_type=="d"):
            try: 
                for i in range(self.dimX-2):
                    ii = (self.dimX*self.dimY) - i - 2
                    self.A[ii][ii] = 1
                    self.b[ii] = T_d
                    
            except:
                print("no T_d value for source boundary type")
        elif (bc_type=="n"):
            try:
                for i in range(self.dimX-2):
                    ii = (self.dimX*self.dimY) - i - 2
                    self.b[ii] = q
                    self.A[ii][ii] = 3.0/(2*self.dx)
                    self.A[ii][ii-self.dimX] = -4.0/(2*self.dx)
                    self.A[ii][ii-(2*self.dimX)] = 1.0/(2*self.dx)
            except:
                print("no q value for flux boundary type")
        elif (bc_type=="r"):
            try:
                for i in range(self.dimX-2):
                    ii = (self.dimX*self.dimY) - i - 2
                    self.b[ii] = alpha * T_inf
                    self.A[ii][ii] = alpha + 3.0/(2*self.dx)
                    self.A[ii][ii-self.dimX] = -4.0/(2*self.dx)
                    self.A[ii][ii-(2*self.dimX)] = 1.0/(2*self.dx)
            except:
                print("no alpha or T_inf value for conjugate boundary type")
        else:
            raise TypeError("Unknown boundary condition: {0:s}".format(bc_type))


    # north
    def set_north(self, bc_type, T_d=0.0, q=0.0, alpha = 0.0, T_inf=0.0):
        if (bc_type=="d"):
            try: 
                for i in range(self.dimX - 2):
                    ii = self.dimX - i - 2
                    self.A[ii][ii] = 1
                    self.b[ii] = T_d
                    
            except:
                print("no T_d value for source boundary type")
        elif (bc_type=="n"):
            try:
                for i in range(1, self.dimX):
                    ii = i
                    self.b[ii] = q
                    self.A[ii][ii] = 3.0/(2*self.dx)
                    self.A[ii][ii+self.dimX] = -4.0/(2*self.dx)
                    self.A[ii][ii+(2*self.dimX)] = 1.0/(2*self.dx)
            except:
                print("no q value for flux boundary type")
        elif (bc_type=="r"):
            try:
                for i in range(1, self.dimX):
                    ii = i
                    self.b[ii] = alpha * T_inf
                    self.A[ii][ii] = alpha + 3.0/(2*self.dx)
                    self.A[ii][ii+self.dimX] = -4.0/(2*self.dx)
                    self.A[ii][ii+(2*self.dimX)] = 1.0/(2*self.dx)
            except:
                print("no alpha or T_inf value for conjugate boundary type")
        else:
            raise TypeError("Unknown boundary condition: {0:s}".format(bc_type))


    # west
    def set_west(self, bc_type, T_d=0.0, q=0.0, alpha = 0.0, T_inf=0.0):
        if (bc_type=="d"):
            try: 
                for i in range(self.dimY):
                    ii = i * self.dimX
                    self.b[ii] = T_d
                    self.A[ii][ii] = 1
            except:
                print("no T_d value for source boundary type")
        elif (bc_type=="n"):
            try:
                for i in range(self.dimY):
                    ii = i * self.dimX
                    self.b[ii] = q
                    self.A[ii][ii] = 3.0/(2*self.dx)
                    self.A[ii][ii+1] = -4.0/(2*self.dx)
                    self.A[ii][ii+2] = 1.0/(2*self.dx)
            except:
                print("no q value for flux boundary type")
        elif (bc_type=="r"):
            try:
                for i in range(self.dimY):
                    ii = i * self.dimX
                    self.b[ii] = alpha * T_inf
                    self.A[ii][ii] = alpha + 3.0/(2*self.dx)
                    self.A[ii][ii+1] = -4.0/(2*self.dx)
                    self.A[ii][ii+2] = 1/(2.0*self.dx)
            except:
                print("no alpha or T_inf value for conjugate boundary type")
        else:
            raise TypeError("Unknown boundary condition: {0:s}".format(bc_type))


# east
    def set_east(self, bc_type, T_d=0.0, q=0.0, alpha = 0.0, T_inf=0.0):
        if (bc_type=="d"):
            try: 
                for i in range(self.dimY):
                    ii = i * self.dimX + self.dimX -1
                    self.b[ii] = T_d
                    self.A[ii][ii] = 1
            except:
                print("no T_d value for source boundary type")
        elif (bc_type=="n"):
            try:
                for i in range(self.dimY):
                    ii = i * self.dimX + self.dimX -1
                    self.b[ii] = q
                    self.A[ii][ii] = 3.0/(2*self.dx)
                    self.A[ii][ii-1] = -4.0/(2*self.dx)
                    self.A[ii][ii-2] = 1.0/(2*self.dx)
            except:
                print("no q value for flux boundary type")
        elif (bc_type=="r"):
            try:
                for i in range(self.dimY):
                    ii = i * self.dimX + self.dimX -1
                    self.b[ii] = alpha * T_inf
                    self.A[ii][ii] = alpha + 3/(2*self.dx)
                    self.A[ii][ii-1] = -4/(2*self.dx)
                    self.A[ii][ii-2] = 1/(2*self.dx)
            except:
                print("no alpha or T_inf value for conjugate boundary type")
        else:
            raise TypeError("Unknown boundary condition: {0:s}".format(bc_type))
       
       

    # solve the linear system
    def solve(self):
        return np.linalg.solve(self.A, self.b)


#### FVM solution

In [ ]:
heat = SteadyHeat2D(Lx, Ly, 50, 50)
heat.set_south("d", Temp)
heat.set_west("d", Temp)
heat.set_north("n", q)
heat.set_east("n", q)

x = heat.solve()
FD_matrix = x.reshape((50, 50))

#### Green solution

In [ ]:
# source surface area
x_s_start = Lx/3
x_s_end = 2*x_s_start
y_s_start = Ly/3
y_s_end = 2*y_s_start

# x_s and y_s array for integral calculation
n_step_s = 50
step_s_x = (x_s_end-x_s_start)/n_step_s
step_s_y = (y_s_end-y_s_start)/n_step_s
x_s = np.linspace(x_s_start, x_s_end, n_step_s)
y_s = np.linspace(y_s_start, y_s_end, n_step_s)

T_field_dirichlet_small = np.zeros((50, 50))
for i in range(len(x)):
    for j in range(len(x)):
        T_field_dirichlet_small[i, j] = HeatEq_2D_point(x[i], y[j], omega, lambda_v, Lx, Ly, x_s_start, x_s_end, y_s_start, y_s_end, "N", "N", "D", "D", Temp, q)

#### Combination

In [ ]:
combination = T_field_dirichlet_small + FD_matrix